# Hands-on AWS - 4 - Video analytics

## Installing dependencies
Here we install relevant dependencies to run serverless framework.

In [ ]:
!pip install awscli --upgrade --user
!npm install -g serverless

## Setting AWS environmental variables
Here we set up AWS environmental variables so that we will be able to deploy to our AWS account. We will need access key id, secret access key and account id. You will need to replace AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_ACCOUNT_ID with your values. Please use test account and temporary credentials or deactivate credentials after usage.

In [ ]:
%env AWS_ACCESS_KEY_ID=<AWS_ACCESS_KEY_ID>
%env AWS_SECRET_ACCESS_KEY=<AWS_SECRET_ACCESS_KEY>
%env AWS_ACCOUNT_ID=<AWS_ACCOUNT_ID>
%env AWS_DEFAULT_REGION=us-west-1
%env HASH=1q2w3e

## Setting environmental variables
Here we set up unique environmental variables which we will be using to name our functions.

In [ ]:
%env UNIQUE_ID=<firstname-lastname>

# Create bucket
add bucket name enivironment variable.

In [ ]:
!aws s3api create-bucket --bucket $UNIQUE_ID --create-bucket-configuration LocationConstraint=us-west-1
!aws s3api delete-bucket --bucket $UNIQUE_ID


# Lambda deployment - object recog

Let's deploy the object recognition lambda.
First we have to create a role which can be assumed by the lambda function.

In [ ]:
!aws iam create-role --role-name lambda-role-$UNIQUE_ID --assume-role-policy-document file://roles/lambda-role.json | jq '.Role.Arn'

export the role variable for later use

In [ ]:
%env LAMBDA_ROLE_ARN=<arn>

delete role in case of typo

In [ ]:
!aws iam  delete-role --role-name lambda-role-$UNIQUE_ID

create policy with the list of permissions for the lambda

In [ ]:
!aws iam create-policy --policy-name lambda-policy-$UNIQUE_ID --policy-document file://roles/lambda-policy.json | jq '.Policy.Arn'

In [ ]:
%env LAMBDA_POLICY_ARN=<arn>

delete role in case of typo

In [ ]:
!aws iam  delete-policy --policy-arn $LAMBDA_POLICY_ARN

Now, we will attach the permissions for the above role.

In [ ]:
!aws iam attach-role-policy --role-name lambda-role-$UNIQUE_ID --policy-arn $LAMBDA_POLICY_ARN

detach in case of typo

In [ ]:
!aws iam detach-role-policy --role-name lambda-role-$UNIQUE_ID --policy-arn $LAMBDA_POLICY_ARN

Create lambda function from ECR image

In [ ]:
!aws lambda create-function --function-name recog-$UNIQUE_ID \
                            --package-type Image \
                            --code ImageUri="705254273855.dkr.ecr.us-west-1.amazonaws.com/video-analytics-recog-aws:latest" \
                            --role $LAMBDA_ROLE_ARN \
                            --timeout 120 \
                            --tracing-config Mode=Active \
                            --publish

delete command in case of typo

In [ ]:
!aws lambda delete-function --function-name recog-$UNIQUE_ID

# Deploy decoder and streaming lambdas

In [ ]:
!./deploy-decoder-Streaming-Lambda.sh

# Lambda invoke - streaming
Invoke Lambda using cli

In [ ]:
!aws lambda invoke --function-name streaming-$UNIQUE_ID \
                   --cli-binary-format raw-in-base64-out \
                   --payload '{ "name": "'$UNIQUE_ID'", "TransferType": "S3" }' response.json

# Lambda response

In [ ]:
!cat ./response.json

# Clean up resources

In [ ]:
!./cleanup.sh